In [0]:
import os
import warnings
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
import pandas as pd


In [2]:
# 구글 드라이브 연동
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
faqs = pd.read_csv(os.path.join('data','/content/drive/My Drive/data/jokes.csv'))
faqs

,ID,Question,Answer
0,1,Did you hear about the Native American man tha...,He nearly drown in his own tea pee.
1,2,What's the best anti diarrheal prescription?,Mycheexarphlexin
2,3,What do you call a person who is outside a doo...,Matt
3,4,Which Star Trek character is a member of the m...,Jean-Luc Pickacard
4,5,What's the difference between a bullet and a h...,A bullet doesn't miss Harambe
...,...,...,...
38264,38265,Q: Why did the pacifist /b/tard try to calm ev...,He did it for the
38265,38266,Q: Why can't Obama poke fun at himself?,A: Because that would be racist.
38266,38267,Why is gambling not allowed in Africa?,Because there are too many cheetahs.
38267,38268,What do you call three witches in a hot tub?,A self-cleaning coven.


In [0]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# 토근화
tokened_questions = [word_tokenize(question.lower()) for question in faqs['Question']]
tokened_questions

In [0]:
lemmatizer = WordNetLemmatizer()

In [8]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
# lemmatization
lemmed_questions = [[lemmatizer.lemmatize(word) for word in doc] for doc in tokened_questions]
lemmed_questions

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
# stopword 제거 불용어 제거하기
stop_words = stopwords.words('english')
questions = [[w for w in doc if not w in stop_words] for doc in lemmed_questions]
questions

In [0]:
# 리스트에서 각 문장부분 토큰화

index_questions = []
for i in range(len(faqs)):
    index_questions.append([questions[i], i ])

# Doc2Vec에서 사용하는 태그문서형으로 변경
tagged_questions = [TaggedDocument(d, [int(c)]) for d, c in index_questions]

In [0]:
tagged_questions

In [0]:
    # make model
import multiprocessing
cores = multiprocessing.cpu_count()
d2v_faqs = doc2vec.Doc2Vec(
                                vector_size=200,
#                                 alpha=0.025,
#                                 min_alpha=0.025,
                                hs=1,
                                negative=0,
                                dm=0,
                                dbow_words = 1,
                                min_count = 5,
                                workers = cores,
                                seed=0,
                                epochs=20
                                )
d2v_faqs.build_vocab(tagged_questions)
d2v_faqs.train(tagged_faqs,
               total_examples = d2v_faqs.corpus_count,
               epochs = d2v_faqs.epochs
                                )
    # # train document vectors
    # for epoch in range(50):
    #     d2v_faqs.train(tagged_faqs,
    #                                total_examples = d2v_faqs.corpus_count,
    #                                epochs = d2v_faqs.epochs
    #                                )
    #     d2v_faqs.alpha -= 0.0025 # decrease the learning rate
    #     d2v_faqs.min_alpha = d2v_faqs.alpha # fix the learning rate, no decay

In [22]:
# 테스트하는 문장도 같은 전처리를 해준다.
test_string = "What's the best anti diarrheal prescription?"
tokened_test_string = word_tokenize(test_string)
lemmed_test_string = [lemmatizer.lemmatize(word) for word in tokened_test_string]
test_string = [w for w in lemmed_test_string if not w in stop_words]

test_string

['What', "'s", 'best', 'anti', 'diarrheal', 'prescription', '?']

In [25]:
topn = 5
test_vector = d2v_faqs.infer_vector(test_string)
result = d2v_faqs.docvecs.most_similar([test_vector], topn=topn)
print(result)

for i in range(topn):
    print("{}위. {}, {} {}".format(i+1, result[i][1], result[i][0],faqs['Question'][result[i][0]] ))

[(1, 0.9341994524002075), (21792, 0.6980049014091492), (29760, 0.6693370342254639), (6473, 0.6332617998123169), (17287, 0.6220482587814331)]
1위. 0.9341994524002075, 1 What's the best anti diarrheal prescription?
2위. 0.6980049014091492, 21792 What's the best angle to tackle a problem?
3위. 0.6693370342254639, 29760 What's a Jackhammer's Best Friend?
4위. 0.6332617998123169, 6473 What's the best way to get to the front page?
5위. 0.6220482587814331, 17287 What's the best thing about having insomnia?


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [27]:
# 성능 측정
raten = 5
found = 0
for i in range(len(faqs)):
    tstr = faqs['Question'][i]
    tokened_test_string = word_tokenize(tstr)
    lemmed_test_string = [lemmatizer.lemmatize(word) for word in tokened_test_string]
    ttok = [w for w in lemmed_test_string if not w in stop_words]
    tvec = d2v_faqs.infer_vector(ttok)
    re = d2v_faqs.docvecs.most_similar([tvec], topn = raten)
    for j in range(raten):
        if i == re[j][0]: 
            found = found + 1
            break

print("정확도 = {} % ({}/{} )  ".format(found/len(faqs),found, len(faqs)))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


정확도 = 0.8619509263372442 % (32986/38269 )  


In [0]:
# 모델 저장

# 모델 1
#d2v_faqs.save(os.path.join('data','/content/drive/My Drive/data/d2v_faqs_size100_min1_batch50_epoch100_nounonly_dm0.model'))

#모델 2
d2v_faqs.save(os.path.join('data','/content/drive/My Drive/data/d2v_faqs_size200_min5_epoch20_jokes.model'))

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
